In [2]:
from collections import Counter
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch import nn, optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import matplotlib.pyplot as plt

In [4]:
import os

if os.path.exists('dataset_eskov.txt'):
    print("YES")


YES


In [8]:
with open('dataset_eskov.txt', encoding="utf8") as f:
    lines = f.readlines()

In [9]:
lines[1]

'Все началось с того, что Дмитрий, прогуливаясь по смотровой площадке, поскользнулся и сорвался с высоты. Он не кричал, не пытался цепляться — он будто смирился с неизбежным. Как рассказывают очевидцы, он падал пластом, словно огромный лист бумаги, распластавшись в воздухе. Сила удара о землю должна была быть разрушительной, но произошло нечто поразительное.\n'

In [10]:
def preprocess(line):
    return ' '.join(w.lower() for w in (''.join(ch for ch in word if ch.isalpha()) for word in line.split()) if w)

In [11]:
preprocess(lines[1])

'все началось с того что дмитрий прогуливаясь по смотровой площадке поскользнулся и сорвался с высоты он не кричал не пытался цепляться он будто смирился с неизбежным как рассказывают очевидцы он падал пластом словно огромный лист бумаги распластавшись в воздухе сила удара о землю должна была быть разрушительной но произошло нечто поразительное'

In [12]:
class Dataset(torch.utils.data.Dataset):
    def __init__(
        self,
        lines,
    ):
        self.lines = lines
        self.pad_token = '<PAD>'
        self.bos_token = '<BOS>'
        self.eos_token = '<EOS>'
        self.uniq_words = [self.pad_token, self.bos_token, self.eos_token] + self.get_uniq_words()

        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        self.pad_token_id = self.word_to_index['<PAD>']
        self.bos_token_id = self.word_to_index['<BOS>']
        self.eos_token_id = self.word_to_index['<EOS>']

        self.tokenized = [[self.word_to_index[w] for w in line.split()] for line in self.lines]

    def get_uniq_words(self):
        word_counts = Counter(word for line in self.lines for word in line.split())
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, index):
        return (
            torch.LongTensor([self.bos_token_id] + self.tokenized[index]),
            torch.LongTensor(self.tokenized[index] + [self.eos_token_id]),
        )

In [13]:
dataset = Dataset([preprocess(line) for line in lines])

In [14]:
dataset.word_to_index

{'<PAD>': 0,
 '<BOS>': 1,
 '<EOS>': 2,
 'что': 3,
 'на': 4,
 'он': 5,
 'дмитрий': 6,
 'и': 7,
 'с': 8,
 'его': 9,
 'не': 10,
 'по': 11,
 'стало': 12,
 'который': 13,
 'в': 14,
 'все': 15,
 'как': 16,
 'очевидцы': 17,
 'но': 18,
 'асфальт': 19,
 'бок': 20,
 'след': 21,
 'говорят': 22,
 'однажды': 23,
 'еськов': 24,
 'известный': 25,
 'своей': 26,
 'неуемной': 27,
 'тягой': 28,
 'к': 29,
 'высоте': 30,
 'приключениям': 31,
 'оказался': 32,
 'вершине': 33,
 'знаменитого': 34,
 'эмпайрстейтбилдинга': 35,
 'легенде': 36,
 'невероятное': 37,
 'падение': 38,
 'городским': 39,
 'мифом': 40,
 'передают': 41,
 'из': 42,
 'уст': 43,
 'уста': 44,
 'началось': 45,
 'того': 46,
 'прогуливаясь': 47,
 'смотровой': 48,
 'площадке': 49,
 'поскользнулся': 50,
 'сорвался': 51,
 'высоты': 52,
 'кричал': 53,
 'пытался': 54,
 'цепляться': 55,
 'будто': 56,
 'смирился': 57,
 'неизбежным': 58,
 'рассказывают': 59,
 'падал': 60,
 'пластом': 61,
 'словно': 62,
 'огромный': 63,
 'лист': 64,
 'бумаги': 65,
 'распл

In [15]:
word_to_index_list = list(dataset.word_to_index.items())
word_to_index_list[:10]

[('<PAD>', 0),
 ('<BOS>', 1),
 ('<EOS>', 2),
 ('что', 3),
 ('на', 4),
 ('он', 5),
 ('дмитрий', 6),
 ('и', 7),
 ('с', 8),
 ('его', 9)]

In [16]:
len(dataset.word_to_index)

144

In [17]:
dataset[0]

(tensor([ 1, 22,  3, 23,  6, 24, 25, 26, 27, 28, 29, 30,  7, 31, 32,  4, 33, 34,
         35, 11, 36,  9, 37, 38, 12, 39, 40, 13, 41, 42, 43, 14, 44]),
 tensor([22,  3, 23,  6, 24, 25, 26, 27, 28, 29, 30,  7, 31, 32,  4, 33, 34, 35,
         11, 36,  9, 37, 38, 12, 39, 40, 13, 41, 42, 43, 14, 44,  2]))

In [18]:
def pad_collate(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x) for x in xx]
    y_lens = [len(y) for y in yy]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=dataset.pad_token_id)
    yy_pad = pad_sequence(yy, batch_first=True, padding_value=dataset.pad_token_id)

    return xx_pad, yy_pad, x_lens, y_lens

In [19]:
dataloader = DataLoader(dataset, batch_size=512, collate_fn=pad_collate, shuffle=True)

In [20]:
class Model(nn.Module):
    def __init__(self, vocab_len):
        super(Model, self).__init__()
        self.hidden_size = 256
        self.embedding_dim = 256
        self.num_layers = 3

        vocab_len = len(dataset.uniq_words)
        self.embedding = nn.Embedding(
            num_embeddings=vocab_len,
            embedding_dim=self.embedding_dim,
            padding_idx=0,
        )
        self.rnn = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            dropout=0.2,
            batch_first=True,
        )
        self.fc = nn.Linear(self.hidden_size, vocab_len)

    def forward(self, x, lens=None, prev_state=None):
        embed = self.embedding(x)
        if lens is None:
            output, state = self.rnn(embed, prev_state)
        else:
            embed_packed = pack_padded_sequence(embed, lens, batch_first=True, enforce_sorted=False)
            output_packed, state = self.rnn(embed_packed, prev_state)
            output, _ = pad_packed_sequence(output_packed, batch_first=True)
        logits = self.fc(output)
        return logits, state

In [21]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = Model(len(dataset.uniq_words)).to(DEVICE).train()

In [22]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

1652880

In [23]:
criterion = nn.CrossEntropyLoss(ignore_index=dataset.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-3)

In [24]:
def train(model, dataloader, criterion, optimizer, epochs):
    losses = []
    model.train()

    for epoch in range(epochs):
        for x, y, x_lens, y_lens in dataloader:
            optimizer.zero_grad()

            y_pred, _ = model(x.to(DEVICE), x_lens)
            loss = criterion(y_pred.transpose(1, 2), y.to(DEVICE))

            loss.backward()
            optimizer.step()

        losses.append(loss.item())
        if epoch % 10 == 0:
            print({ 'epoch': epoch, 'loss': losses[-1] })

    return losses

In [25]:
loss_history = train(model, dataloader, criterion, optimizer, epochs=150)

{'epoch': 0, 'loss': 4.972452640533447}
{'epoch': 10, 'loss': 2.0977635383605957}
{'epoch': 20, 'loss': 1.0917316675186157}
{'epoch': 30, 'loss': 0.6819555759429932}
{'epoch': 40, 'loss': 0.4479215443134308}
{'epoch': 50, 'loss': 0.28411930799484253}
{'epoch': 60, 'loss': 0.1914079636335373}
{'epoch': 70, 'loss': 0.11786799132823944}
{'epoch': 80, 'loss': 0.08593345433473587}
{'epoch': 90, 'loss': 0.0722772404551506}
{'epoch': 100, 'loss': 0.05832066759467125}
{'epoch': 110, 'loss': 0.05234261974692345}
{'epoch': 120, 'loss': 0.046354662626981735}
{'epoch': 130, 'loss': 0.04339760169386864}
{'epoch': 140, 'loss': 0.044218238443136215}


In [26]:
def tokenize(value):
    return [dataset.bos_token_id]+[dataset.word_to_index[word.lower()] for word in value.split()]


def decode(token_ids):
    return ' '.join(dataset.index_to_word[token_id] for token_id in token_ids)


@torch.no_grad()
def generate(prompt, max_tokens=20):
    model.eval()
    response = []
    state = None
    prompt_tokens = tokenize(prompt)
    model_input = torch.LongTensor([prompt_tokens]).to(DEVICE)
    for _ in range(max_tokens):
        logits, state = model(model_input, prev_state=state)
        token_argmax = logits[0, -1].argmax()
        response.append(token_argmax.item())
        if response[-1] == dataset.eos_token_id:
            break
        model_input = token_argmax.view(1, 1)

    return decode(prompt_tokens + response)

In [28]:
generate('еськов')

'<BOS> еськов что дмитрий еськов известный своей неуемной тягой к высоте и приключениям оказался на вершине знаменитого эмпайрстейтбилдинга по легенде его невероятное'

In [29]:
@torch.no_grad()
def sample(prompt, max_tokens=20):
    model.eval()
    response = []
    state = None
    prompt_tokens = tokenize(prompt)
    model_input = torch.LongTensor([prompt_tokens]).to(DEVICE)
    for _ in range(max_tokens):
        logits, state = model(model_input, prev_state=state)
        token_probs = F.softmax(logits[0, -1], dim=-1).cpu().numpy()
        sampled_token = np.random.choice(len(token_probs), p=token_probs)
        response.append(sampled_token)
        if response[-1] == dataset.eos_token_id:
            break
        model_input = torch.LongTensor([[sampled_token]]).to(DEVICE)

    return decode(prompt_tokens + response)

In [33]:
sample('еськов')

'<BOS> еськов что дмитрий дмитрий известный своей неуемной тягой к высоте и приключениям оказался на вершине знаменитого эмпайрстейтбилдинга по легенде его невероятное'

In [39]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/ruT5-large")

ValueError: Converting from Tiktoken failed, if a converter for SentencePiece is available, provide a model path with a SentencePiece tokenizer.model file.Currently available slow->fast convertors: ['AlbertTokenizer', 'BartTokenizer', 'BarthezTokenizer', 'BertTokenizer', 'BigBirdTokenizer', 'BlenderbotTokenizer', 'CamembertTokenizer', 'CLIPTokenizer', 'CodeGenTokenizer', 'ConvBertTokenizer', 'DebertaTokenizer', 'DebertaV2Tokenizer', 'DistilBertTokenizer', 'DPRReaderTokenizer', 'DPRQuestionEncoderTokenizer', 'DPRContextEncoderTokenizer', 'ElectraTokenizer', 'FNetTokenizer', 'FunnelTokenizer', 'GPT2Tokenizer', 'HerbertTokenizer', 'LayoutLMTokenizer', 'LayoutLMv2Tokenizer', 'LayoutLMv3Tokenizer', 'LayoutXLMTokenizer', 'LongformerTokenizer', 'LEDTokenizer', 'LxmertTokenizer', 'MarkupLMTokenizer', 'MBartTokenizer', 'MBart50Tokenizer', 'MPNetTokenizer', 'MobileBertTokenizer', 'MvpTokenizer', 'NllbTokenizer', 'OpenAIGPTTokenizer', 'PegasusTokenizer', 'Qwen2Tokenizer', 'RealmTokenizer', 'Refor

In [36]:
len(dataset.uniq_words)

144

In [37]:
len(set(
    x for tokens in (
        tokenizer(
            preprocess(line),
            add_special_tokens=False,
        )['input_ids'] for line in lines
    ) for x in tokens
))

NameError: name 'tokenizer' is not defined